In [2]:
import requests
import time
import json
from bs4 import BeautifulSoup
import xlsxwriter,xlrd
from xlutils.copy import copy
import urllib.parse

In [3]:
# 生成Excel

def generate_excel(f_re):

    file_name = './运营产品.xlsx'

    try:#追加写入
        data = xlrd.open_workbook(file_name)  # 打开fname文件
        sheets = data.sheet_names()  # 获取工作簿中的所有表格
        worksheet2 = data.sheet_by_name(sheets[0])  # 获取工作簿中所有表格中的的第一个表格
        rows_old = worksheet2.nrows  # 获取表格中已存在的数据的行数
        new_workbook = copy(data)  # 将xlrd对象拷贝转化为xlwt对象
        new_worksheet = new_workbook.get_sheet(0)

        for j in range(0,len(list(f_re.values()))):
            new_worksheet.write(rows_old, j, str(list(f_re.values())[j]).strip('/'))  # 追加写入数据
        new_workbook.save(file_name)  # 保存工作簿
        print("xls格式表格【追加】写入数据成功！")
        return
    except Exception as e:
        print(e)

    workbook = xlsxwriter.Workbook(file_name)#首次写入 
    worksheet = workbook.add_worksheet()

    # 设定格式，等号左边格式名称自定义，字典中格式为指定选项
    # bold：加粗，num_format:数字格式
    bold_format = workbook.add_format({'bold': True})
    # money_format = workbook.add_format({'num_format': '$#,##0'})
    # date_format = workbook.add_format({'num_format': 'mmmm d yyyy'})

    # 将二行二列设置宽度为15(从0开始)
    worksheet.set_column(1, 1, 15)

    # 用符号标记位置，例如：A列1行
    worksheet.write('A1', 'position_id', bold_format)
    worksheet.write('B1', 'position_name', bold_format)
    worksheet.write('C1', 'company_name', bold_format)
    worksheet.write('D1', 'benefits', bold_format)
    worksheet.write('E1', 'education', bold_format)
    worksheet.write('F1', 'type', bold_format)
    worksheet.write('G1', 'years', bold_format)
    worksheet.write('H1', 'publish_time', bold_format)
    worksheet.write('I1', 'city', bold_format)
    worksheet.write('J1', 'salary', bold_format)
    worksheet.write('K1', 'job_detail', bold_format)
    col = 0
    nrows = 1
    worksheet.write_string(nrows, col, str(f_re['position_id']))
    worksheet.write_string(nrows, col + 1, str(f_re['position_name']))
    worksheet.write_string(nrows, col + 2, str(f_re['company_name']))
    worksheet.write_string(nrows, col + 3, str(f_re['benefits']))
    worksheet.write_string(nrows, col + 4, str(f_re['education']).strip('/'))
    worksheet.write_string(nrows, col + 5, str(f_re['type']).strip('/'))
    worksheet.write_string(nrows, col + 6, str(f_re['years']).strip('/'))
    worksheet.write_string(nrows, col + 7, str(f_re['publish_time']))
    worksheet.write_string(nrows, col + 8, str(f_re['city']).strip('/'))
    worksheet.write_string(nrows, col + 9, str(f_re['salary']))
    worksheet.write_string(nrows, col + 10, str(f_re['job_detail']))

    workbook.close()
    return

In [6]:
# 请求一个职位

def requestJob(position_id,times,url_start,job,city,pageNum):

    url_parse = "https://www.lagou.com/jobs/"+str(position_id)+".html"#构造造详情页链接
    # 构造请求头
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Referer': url_start,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
    }

    s = requests.Session()
    # 请求首页获取cookies
    s.get(url_start, headers=headers, timeout=10)  
    # 为此次获取的cookies
    cookie = s.cookies  
    
    # 获取此次文本
    response = s.post(url_parse, headers=headers, cookies=cookie, timeout=10)  
    
    
    # 使用临时文件的方式来保存，然后对临时文件进行处理
    f = open('./test.html',mode='w')
    try:
        f.write(response.text)#写入html文件
    except:
        print("html 写入失败")
        return False
    f.close()
    # 使用临时文件的方式来保存，然后对临时文件进行处理
    
    
    
    #读取解析html文件
    f2 = open('./test.html') #如果直接传f句柄进入BeautifulSoup读不到内容
    
    
    soup = BeautifulSoup(f2, 'html.parser')
    
    f2.close()
    #读取解析html文件
    
    
    
    #定义一个列表收集我们所需要的信息
    result = [] 
    #定义一个字典用来收集我们所需要的信息，然后用于写入Excel
    f_re = {} 
    # ![](http://ww3.sinaimg.cn/large/006tNc79ly1g3rhky13lvj31hn0u07pg.jpg)
    for val in soup.find_all('span'):
        if val != None:
            result.append(val.string)
    try:
        f_re['position_id'] = position_id;
        f_re['city'] = result[8];
        f_re['company_name'] = soup.find(attrs={"class": "fl-cn"}).text.strip()
        f_re['benefits'] = soup.find(attrs={"class": "advantage"}).next_element.next_element.next_element.string
        f_re['education'] = result[10]
        f_re['type'] = result[11]
        f_re['years'] = result[9]
        f_re['publish_time'] = soup.find(attrs={"class": "publish_time"}).text.strip()
        f_re['position_name'] = result[6]
        f_re['salary'] = result[7]
        f_re['job_detail'] = soup.find(attrs={"class": "job_bt"}).text

        #写入Excel文件
        generate_excel(f_re)
        print(f_re)
        print('\n')

        # if(times == 2):
        #     exit()

        print('all===data====job==='+job+'city===='+city+'==pageNum='+str(pageNum)+'==num===='+str(times))
        print('\n')
    except:
        print("请求异常")
        print(response.text)
        return False
    time.sleep(5)

    return True

In [4]:
def main():

    job_list = ['数据分析','产品经理','Java','PHP','C++']
    
    city_list = ['全国'] #如果是去全国站则 ["全国"] 即可
    
    pageNum = 1 #页数

    for job in job_list:
        for city in city_list:

            times = 1
            # url_start = "https://www.lagou.com/jobs/list_%E6%95%B0%E6%8D%AE%E5%88%86%E6%9E%90?labelWords=&fromSearch=true&suginput="
            
            ## 浏览器上能看到的地址
            url_start = "https://www.lagou.com/jobs/list_"+urllib.parse.quote(job)+"?labelWords=&fromSearch=true&suginput="
            
            ## 真正包含了 job 信息的地址
            url_parse = "https://www.lagou.com/jobs/positionAjax.json?city="+urllib.parse.quote(city)+"&needAddtionalResult=false"
            
            # ![](http://ww4.sinaimg.cn/large/006tNc79ly1g3rh8umeglj31f20u0h4r.jpg)
            
            # 请求头
            headers = {
                'Accept': 'application/json, text/javascript, */*; q=0.01',
                'Referer': url_start,
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
            }


            for currentPage in range(pageNum, 500):
                # 请求参数
                data = {
                    'first': 'true',
                    'pn': str(currentPage),
                    'kd': job
                        }
                
                if currentPage != 1 :
                    data['first'] = 'false'

                ## 生成一个请求的 Session 对象
                ## 关于请求的 cookie 相关的配置都在这里
                ## 关于请求的发起都在这里
                requestsSession = requests.Session()
                
                # 请求首页获取cookies
                requestsSession.get(url_start, headers=headers, timeout=10)  
                
                # 为此次获取的cookies
                cookie = requestsSession.cookies  
                
                
                response = requestsSession.post(url_parse, data=data, headers=headers, cookies=cookie, timeout=10)  # 获取此次文本 获取列表页

                response.encoding = response.apparent_encoding
                text = json.loads(response.text)

                if (text['code'] != 0):
                    print(response.text)
                    exit("请求异常")
                # http://ww3.sinaimg.cn/large/006tNc79ly1g3rhatl6o9j31k40u0tu6.jpg
                info = text["content"]["positionResult"]["result"]

                if(len(info) == 0):
                    pageNum = 1
                    print('\n')
                    print("========================NO+DATA=================================")
                    break

                #遍历十五条职位信息，拿到positionID 然后去请求职位详情页
                for singleJobInfo in info:

                    position_id = singleJobInfo['positionId']
                    #请求详情页
                    res = requestJob(position_id,times,url_start,job,city,currentPage)
                    if(res == True):
                        times += 1

In [7]:
main()

[Errno 2] No such file or directory: './运营产品.xlsx'
{'position_id': 5944929, 'city': '/北京 /', 'company_name': '玖富集团', 'benefits': '公司牛人多', 'education': '本科及以上 /', 'type': '全职', 'years': '经验不限 /', 'publish_time': '2019-05-17\xa0 发布于拉勾网', 'position_name': '数据分析师', 'salary': '15k-28k ', 'job_detail': '\n职位描述：\n\n        \xa0\n岗位职责：\n1.负责事业部成本控制和预算管理工作，及时预警成本节超情况；\n2.负责撰写专题性数据分析报告，通过数据分析发现产品或业务的关键问题和特征，为产品或运营决策提供合理化建议；\n3.搭建指标监控体系，周期性分析业务关键指标动向；\n4.负责运营活动指标监控、效果追踪及复盘等工作；\n5.及时响应产品、运营、市场的数据统计和分析需求，为精细化运营提供支持。\n\n任职要求：\n1.本科及以上学历，数学系、统计学、信息与计算科学等相关专业；\n2.对数据敏感，有较强的数据分析逻辑，熟练掌握SQL、SPSS等数据分析工具，熟练使用PPT、office、Excel等办公软件；\n3.有2年相关数据分析工作经验，具备处理和分析较大量级数据的能力；能够对平台各类数据、网站流量数据、用户数据、以及转化数据等清晰统计分析；\n4.熟悉数据分析与数据挖掘理论，互联网应用技术知识、网络知识，了解金融平台数据分析统计，了解互联网营销。\n5.具备较强的数据报表、数据报告制作能力。有用户相关分析经验及建模经验优先\n\n'}


all===data====job===数据分析city====全国==pageNum=1==num====1


xls格式表格【追加】写入数据成功！
{'position_id': 3868326, 'city': '/深圳 /', 'company_name': 'OPPO', 'benefits': '核心事业群 千万级日活 氛围好 组织扁平', 'education': '本科及以上 /', 'type':

xls格式表格【追加】写入数据成功！
{'position_id': 5954722, 'city': '/佛山 /', 'company_name': '碧桂园集团数字化管理中心', 'benefits': '500强企业 发展好', 'education': '本科及以上 /', 'type': '全职', 'years': '经验5-10年 /', 'publish_time': '15:28\xa0 发布于拉勾网', 'position_name': '数据分析师', 'salary': '8k-15k ', 'job_detail': '\n职位描述：\n\n一、岗位职责\n1.负责挖掘需求场景并制定价值交付方案，包括根据业务需求，构建业务指标及分析体系辅助决策，并基于业务规律和数据探索构建数据预测模型，建立基于业务场景的端到端数据流程应用方案。 \xa0 \xa02.负责产品线数据分析与运营，包括持续跟踪和监控业务指标表现，并根据业务痛点进行专项分析，常态化数据运营，为决策、管理提供数据支持。 \xa0 \xa03.负责开发和持续完善公司各项业务数据的统计分析模型，确保分析模型的准确性、实用性及可衡量性。 \xa0 \xa04.负责数据产品的价值交付与过程管理，包括制定项目计划、把控方案、协调资源、跟进进度，确保产品质量和过程合规性，高效完成交付实现预期价值。 \xa0 \xa0\n\n二、任职要求\n1、本科及以上学历，5年以上专业工作经验；2、熟悉地产业务，有顾问咨询经验优先；3、有数据分析、数据挖掘等相关工作经验，能独立完成数据清洗、数据分析、数据可视化的全过程，有独立数据分析项目经验；熟悉数据挖掘模型的特征工程、模型调参、模型评估、落地优化的全过程；4、熟悉企业管理的相关方法和实践，具有财务管理、营销管理、成本管理、投资管理等管理经验；5、具有卓越的项目计划、组织、领导、控制能力，能够通过出色的项目管理来达成预定的项目目标；6、极强的学习能力，良好的沟通、协调能力，善于团队协作，做事主动，责任心强，执行力强，抗压能力强；7、熟练使用SPSS/Tableau/Powerbi/帆软BI，有R, Python经验优先。 \xa0 \xa0\n\n'}


all===data====job===数据分析city====全国==pageNum=1==nu

xls格式表格【追加】写入数据成功！
{'position_id': 5959158, 'city': '/深圳 /', 'company_name': '华策', 'benefits': '五险一金,年终奖,通讯补贴,绩效奖金', 'education': '大专及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '10:38\xa0 发布于拉勾网', 'position_name': '数据分析师', 'salary': '10k-15k ', 'job_detail': '\n职位描述：\n\n该岗位为项目外派驻岗，到深圳福田区金田路4036号荣超大厦上班（平安证券），介意慎投\n\n岗位职责：\n1、支持团队数据分析/挖掘需求；\n2、支持团队商机项目开展，提供商机数据，并追踪实际效果；\n3、上级安排的其他工作。\n\n任职要求：\n1、大专以上学历，计算机、金融或统计学专业；有3年或以上数据分析相关工作经验；\n2、 熟练SQL语言，以及使用Excel和PPT制作分析报告；\n3、沟通能力强，有良好的编码习惯，逻辑严谨，思维清晰；\n4、为人稳重、责任心强、细心、好学、接受新知识的能力强、有较强的抗压能力；\n5、证券行业工作经历更佳。\n\n'}


all===data====job===数据分析city====全国==pageNum=2==num====23


xls格式表格【追加】写入数据成功！
{'position_id': 5957659, 'city': '/北京 /', 'company_name': '钱方好近', 'benefits': '全球移动支付建设者的平台', 'education': '本科及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '14:59\xa0 发布于拉勾网', 'position_name': '数据分析师', 'salary': '10k-20k ', 'job_detail': '\n职位描述：\n\n职位描述：\n1、充分理解公司战略目标和运营过程，对公司的产品和业务有全面的认识；\n2、统筹搭建经营分析框架、落实数据分析、得出业务洞察结论，并逐步进行精细化迭代\n3

xls格式表格【追加】写入数据成功！
{'position_id': 5956008, 'city': '/上海 /', 'company_name': 'OYO', 'benefits': '包晚餐；弹性工作', 'education': '本科及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '3天前\xa0 发布于拉勾网', 'position_name': '数据分析师', 'salary': '15k-25k ', 'job_detail': '\n职位描述：\n\n岗位职责：1、整理线下数据报表，并进行汇总分析；2、整理数据报表，并进行定期发送；3、铜鼓数据变化趋势，识别原因，并与相关部分沟通确认，推动跨部门的问题解决；4、定期跟进改善对策的进展，并整理汇总5、梳理业务大数据自动化需求，并与大数据团队沟通，推进数据分析和展示的系统自动化。岗位要求：1、本科以上学历，3年以上工作经验；2、熟悉Excel的函数操作；3、熟悉使用VBA编写宏函数，进行Excel的数据分析；4、熟悉tableau、SQL大数据分析工具优先；5、做事认真负责，善于沟通，责任心强，抗压能力强\n\n'}


all===data====job===数据分析city====全国==pageNum=3==num====35


xls格式表格【追加】写入数据成功！
{'position_id': 5279194, 'city': '/北京 /', 'company_name': '水滴互助', 'benefits': '年底双薪,股票期权,扁平管理,晋升快', 'education': '本科及以上 /', 'type': '全职', 'years': '经验1-3年 /', 'publish_time': '3天前\xa0 发布于拉勾网', 'position_name': '数据分析师', 'salary': '10k-20k ', 'job_detail': '\n职位描述：\n\n工作职责：\n\xa0 1. 配合业务产品进行日常数据分析，提出针对性建议，指导产品优化。分析包括但不限于产品功能评估、用户行为分析、运营活动评估等\n\xa0 2. 搭建适用于各场景下的有效的数据评估框架、数据分析模型、异常报警机制

xls格式表格【追加】写入数据成功！
{'position_id': 4703938, 'city': '/宁波 /', 'company_name': '草料二维码', 'benefits': '扁平化管理、发展空间大、团队氛围好', 'education': '本科及以上 /', 'type': '实习', 'years': '经验应届毕业生 /', 'publish_time': '09:58\xa0 发布于拉勾网', 'position_name': '数据分析实习生', 'salary': '5k-10k ', 'job_detail': '\n职位描述：\n\n你需要做的是：\xa0\n1、创新性思维，挖掘业务核心痛点，用数据的力量，技术的力量建设数据产品化的客户中心；\n2、快速并充分理解业务，搭建有效的自动化监控及可视化体系，用数据来体现业务工作的水平，建立分析指标体系；\n3、确保各指标、各数据的准确性及有效性，能及时定位并修复异常数据问题；\n4、以数据驱动业务，挖掘业务策略和流程的可优化点，提供有效的数据分析结果作为比对支持；\n5、与业务部门、产品部门紧密合作，推进草料二维码相关产品的服务体系和系统的建设，在实际业务中进行验证，用结果去影响公司产品在服务体系和系统建设；\n我们希望你是：\n1、本科及以上学历，2019届毕业生；\n2、对草料产品有了解，同时具备良好的逻辑思维；\n3、高度的责任心，优秀的沟通能力和团队协作能力；\n4、热爱互联网，喜欢钻研；\xa0\n\n'}


all===data====job===数据分析city====全国==pageNum=3==num====45


xls格式表格【追加】写入数据成功！
{'position_id': 4499800, 'city': '/深圳 /', 'company_name': 'OPPO', 'benefits': '优厚福利,海景办公', 'education': '本科及以上 /', 'type': '全职', 'years': '经验1-3年 /', 'publish_time': '1天前\xa0 发布于拉勾网', 'position_name': '数据分析助理', 'salary': '5k-10k ', 'job_detail': '\n职位描述：\n\n

...

xls格式表格【追加】写入数据成功！
{'position_id': 5908425, 'city': '/杭州 /', 'company_name': '中国婚博会', 'benefits': '周末双休,五险一金,弹性工作', 'education': '本科及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '14:54\xa0 发布于拉勾网', 'position_name': '资深数据分析师', 'salary': '20k-35k ', 'job_detail': '\n职位描述：\n\n岗位职责：1、构建分析体系、报表体系和指标体系，并根据业务发展和需求分析情况进行维护和优化；2、构建辅助分析用的数据产品、分析模型，不断提升分析效能和分析模版的复用性；3、熟练使用数据仓库，灵活应用分析工具和方法，进行可靠的趋势判断和预测、归因；4、深入理解公司战略，通过商业项目的需求调研、数据分析、数据挖掘等，为管理层提供积极的数据反馈和决策支持；5、能够结构化、可视化编写商业数据分析报告，洞察数据背后的规律和问题，为运营产品推广等业务决策和动作提供支持；6、开展行业和竞争对手分析，定期提交行业及竞争对手分析报告。职位要求：1、本科及以上学历，数学、统计等专业优先；3年以上数据分析经验，互联网经历优先；2、熟练应用统计分析方法，如聚类分析、主成分分析、决策树分析、回归分析等；3、熟悉sql/python/R，擅长excel及PPT，了解SPSS/SAS等，掌握1-2个可视化工具；4、数据和商业市场敏感，能主动发现和识别业务问题，进行项目分析和业务优化应用落地；5、面对海量需求，能以结果为导向，准确识别判断需求重要性和优先级，合理规划分析计划；6、具有较强的沟通协调、项目推进、产品思维、逻辑思维、数据可视化、数据解读和业务洞察能力；7、为人踏实、勤奋，具有较强的稳定性；\n8、自驱力、创新和执行力强；9、大项目的经验，有丰富的跨团队、部门的项目资源整合能力，能够独立开展研究项目；10、有数据模型建立和数据化运营经验、数据类产品类规划经验。 \xa0 \xa0\xa0备注: 能接受短期的出差\n\n'}


all===data====job===数据分析city====全国==pageNum=42==n

xls格式表格【追加】写入数据成功！
{'position_id': 4574609, 'city': '/上海 /', 'company_name': '虎扑', 'benefits': '平台广阔,团队靠谱,激励优厚', 'education': '本科及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '10:12\xa0 发布于拉勾网', 'position_name': '数据分析工程师', 'salary': '15k-30k ', 'job_detail': '\n职位描述：\n\n岗位职责：\n1）与业务相关方进行需求沟通并独立承担基础数据分析等数据工作；\n2）对日常运营产品数据监控和分析；\n\xa0\n岗位要求：\n1）本科及以上学历，2年以上互联网行业相关经验；\n2）精通sql，熟悉mysql数据库，能编写较复杂的计算逻辑；\n3）熟练运用R、PYTHON、SAS等主流数据分析软件工具；\n4）有数据敏感度，能从海量数据中提取核心结论，并用清晰且简洁的方式呈现其商业洞察；\n5）逻辑思维清晰，严谨细致，有良好的沟通理解能力和跨团队协作精神；\n6）了解Tableau、帆软等BI工具者优先；\n\n'}


all===data====job===数据分析city====全国==pageNum=43==num====632


xls格式表格【追加】写入数据成功！
{'position_id': 5482819, 'city': '/上海 /', 'company_name': 'ZRobot', 'benefits': '五险一金,补充医疗,节日福利,带薪年假', 'education': '本科及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '09:37\xa0 发布于拉勾网', 'position_name': '初级数据分析师-数据分析与...', 'salary': '20k-40k ', 'job_detail': '\n职位描述：\n\n岗位职责：\n1、基于互联网/支付/运营商等各类海量数据，进行数据的清洗、指标加工，分析提取用户行为特征；2、开发各类信用、风险控制、用户价值模型及各类用户画像

xls格式表格【追加】写入数据成功！
{'position_id': 5712371, 'city': '/上海 /', 'company_name': 'OYO', 'benefits': '弹性上下班', 'education': '学历不限 /', 'type': '全职', 'years': '经验不限 /', 'publish_time': '2天前\xa0 发布于拉勾网', 'position_name': '数据科学-数据分析', 'salary': '20k-40k ', 'job_detail': '\n职位描述：\n\n 职位职责： \n\n 1、负责商业化指标体系建设和完善，通过对业务的敏锐洞察和理解，构建合理的评价体系，衡量业务发展情况； \n\n 2、负责商业化BI产品的建设，独立完成产品调研、方案设计以及协调各方资源推动产品上线； \n\n 3、能根据实际业务分析需求，迅速搭建业务监控体系，挖掘流量、产品、策略方面的商业变现机会，驱动商业化业务发展； \n\n 4、及时关注业界内BI类产品以及数据可视化工具的动态， 提供更高效、更完善的BI产品支持分析决策。 \n\n\n 职位要求： \n\n 1、计算机、统计学、数学、经济学等相关专业，扎实的机器学习或数据挖掘理论和技术基础，具有相关领域较深入的技术应用的经验和能力； \n\n 2、有BI产品建设经验，对用户体验敏感； \n\n 3、良好的沟通能力、团队合作精神、工作规划能力和主动意识； \n\n 4、良好的逻辑思维能力、业务解读能力和快速学习能力，能够独立领导完整的项目； \n\n\n\n\n'}


all===data====job===数据分析city====全国==pageNum=43==num====644


xls格式表格【追加】写入数据成功！
{'position_id': 5823626, 'city': '/深圳 /', 'company_name': '每日优鲜', 'benefits': '带薪年假、定期体检、弹性工作', 'education': '本科及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '2019-05-13\xa0 发布于拉勾网', 'position_name': '数据分

xls格式表格【追加】写入数据成功！
{'position_id': 5943812, 'city': '/北京 /', 'company_name': '好未来', 'benefits': '上市公司；免费晚餐；免费水果零食', 'education': '本科及以上 /', 'type': '全职', 'years': '经验3-5年 /', 'publish_time': '1天前\xa0 发布于拉勾网', 'position_name': '好未来 数据分析师', 'salary': '20k-40k ', 'job_detail': '\n职位描述：\n\n岗位职责：1、根据业务的需要设计数据采集方案，和团队一起完成相关数据的收集、清洗、入库等工作；2、负责BI数据库的规划及脚本开发与执行；并基于现有数据进行挖掘工作3、基于现有的BI数据库，支持各部门的数据分析需求；4、专题项目和日常运营数据支持和分析，并提供数据分析报告和洞察任职要求：1、5年以上经验，具备良好的沟通表达能力，有责任心、抗压能力；2、熟悉Python、sql等工具，有一定的数据挖掘经验；3、熟悉APP运营的日常工作，具有一定的业务经验；4、熟悉Google analytics优先；\n\n'}


all===data====job===数据分析city====全国==pageNum=44==num====656




ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))